In [1]:
import firedrake
import icepack
import geopandas as gpd
import pyproj

### Read Gulkana glacier outline file with json.load()

In [2]:
import json
outline_filename = "gulkana_outline.geojson"
with open(outline_filename,'r') as geojson_file:
    geojson_data = json.load(geojson_file)

In [7]:
geojson_data

{'type': 'FeatureCollection',
 'name': 'gulkana_outline',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'line_type': 'glac_bound',
    'anlys_id': 401875.0,
    'glac_id': 'G214576E63274N',
    'anlys_time': '2015-07-16T00:00:00',
    'area': 17.567,
    'db_area': 17.5667,
    'width': 0.0,
    'length': 0.0,
    'primeclass': 0.0,
    'min_elev': 1162.0,
    'mean_elev': 0.0,
    'max_elev': 2438.0,
    'src_date': '2009-08-04T00:00:00',
    'rec_status': 'okay',
    'glac_name': 'Gulkana Glacier',
    'wgms_id': 'None',
    'local_id': '50009322',
    'glac_stat': 'exists',
    'gone_date': 'None',
    'gone_dt_e': 0.0,
    'subm_id': 624.0,
    'release_dt': '2015-07-16T00:00:00',
    'proc_desc': 'Semi-automated glacier classification.; From Randolph Glacier Inventory Version 5.0. Source file: 01_rgi50_Alaska.zip.  See http://www.glims.org/RGI/.  GLIMS IDs were adjusted and metadata was adde

### Reproject the geojson lat/lon coordinate to CRS coordinate using geopandas

In [57]:
# source CRS (WGS 84)
source_crs = pyproj.CRS('EPSG:4326')
# target CRS (UTM zone 8 - EPSG:32608)
target_crs = pyproj.CRS('EPSG:32608')
# create a transformer to convert from source to target CRS
transformer = pyproj.Transformer.from_crs(source_crs, target_crs)

In [88]:
def convert_coordinates_to_utm(coordinates):
    # create a transformer to convert from source to target CRS
    transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
    utm_coordinates = [transformer.transform(lon, lat) for lon, lat in coordinates]
    return utm_coordinates

In [90]:
if geojson_data['type'] == 'FeatureCollection':
    features = geojson_data['features']
    #print(features)
    for feature in features:
        geometry_type = feature['geometry']['type']
        print(geometry_type)
        if geometry_type == 'MultiLineString':
            coordinates = feature['geometry']['coordinates']
            #print(coordinates)
            #utm_coodinates = []
            #for line_coordinates in coordinates:
            utm_coodinates = convert_coordinates_to_utm(line_coordinates)
            
            for utm_coodinate in utm_coodinates:
                utm_easting, utm_northing = utm_coodinate
                print("UTM Easting", utm_easting)

MultiLineString


ValueError: too many values to unpack (expected 2)

In [85]:
#feature['geometry']['type']
#feature['geometry']['coordinates']

In [45]:
geojson_data['features']

[{'type': 'Feature',
  'properties': {'line_type': 'glac_bound',
   'anlys_id': 401875.0,
   'glac_id': 'G214576E63274N',
   'anlys_time': '2015-07-16T00:00:00',
   'area': 17.567,
   'db_area': 17.5667,
   'width': 0.0,
   'length': 0.0,
   'primeclass': 0.0,
   'min_elev': 1162.0,
   'mean_elev': 0.0,
   'max_elev': 2438.0,
   'src_date': '2009-08-04T00:00:00',
   'rec_status': 'okay',
   'glac_name': 'Gulkana Glacier',
   'wgms_id': 'None',
   'local_id': '50009322',
   'glac_stat': 'exists',
   'gone_date': 'None',
   'gone_dt_e': 0.0,
   'subm_id': 624.0,
   'release_dt': '2015-07-16T00:00:00',
   'proc_desc': 'Semi-automated glacier classification.; From Randolph Glacier Inventory Version 5.0. Source file: 01_rgi50_Alaska.zip.  See http://www.glims.org/RGI/.  GLIMS IDs were adjusted and metadata was added at NSIDC.',
   'rc_id': 761.0,
   'geog_area': 'Randolph Glacier Inventory; Umbrella RC for merging the RGI into GLIMS',
   'chief_affl': 'University of Colorado',
   'conn_lvl'

In [86]:
# create a GeoDataFrame from the modified GeoJSON data
gdf = gpd.GeoDataFrame.from_features(geojson_data["features"])
print(gdf)

                                            geometry   line_type  anlys_id   
0  MULTILINESTRING Z ((-145.42389 63.30139 0.0000...  glac_bound  401875.0  \

          glac_id           anlys_time    area  db_area  width  length   
0  G214576E63274N  2015-07-16T00:00:00  17.567  17.5667    0.0     0.0  \

   primeclass  ...  term_type  gtng_o1reg  gtng_o2reg rgi_gl_typ loc_unc_x   
0         0.0  ...        9.0         1.0         2.0         91        50  \

  loc_unc_y glob_unc_x glob_unc_y      submitters   
0        50         80         80  Cogley, Graham  \

                                            analysts  
0  Beedle, Matthew; Berthier, Etienne; Bolch, Tob...  

[1 rows x 39 columns]
